# Classifying YouTube Videos for Humpback Whale Encounters - Keras CNN-RNN

In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import pickle
import glob
import cv2
import os

from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns

import wandb

In [4]:
#ngc workspace path (where we keep our data)
workspace_path = '/mount/data'

In [ ]:
#start wandb session for metric logging
wandb.login() 

wandb.init(project="whale-classification-inception")

# Inception V3 (CNN-RNN) 

## Hyperparameters

In [3]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_NUM_FRAMES = 500
NUM_FEATURES = 2048

In [11]:
#read all frames in for 1 video from workspace 'frames' directory
frames = []
for i in range(461):
    if i % 100 == 0:
        print(i)
    
    #read in .jpg file as array for video clip 0000
    img = cv2.imread(workspace_path + f'/frames/clip_0000_frame_{i}.jpg')
    frames.append(img)

frames = np.array(frames)
frames.shape

(461, 224, 224, 3)

461 frames of size 224 x 224 with RGB color channels

# Load Frames + Extract Features

In [ ]:
from feature_extraction import load_frames, prepare_all_videos